# becquerel fitting example

In [ ]:
import becquerel as bq
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'white'

%load_ext autoreload
%autoreload 2

np.random.seed(0)

Read an example spectrum into a Spectrum object

In [ ]:
spec1 = bq.Spectrum.from_file('../tests/samples/digibase_5min_30_1.spe')

Plot the entire spectrum and the ROI we want to fit

In [ ]:
spec1.plot(yscale='log')
plt.show()

spec1.plot(yscale='log', xlim=(350, 450))
plt.show()

Let's first try a Gaussian peak shape and a simple linear background

In [ ]:
model = (bq.fitting.GaussModel(prefix='gauss0_') + bq.fitting.LineModel(prefix='linear_'))

Pass our model and data to a Fitter object, then perform the fit

In [ ]:
fitter = bq.Fitter(
    model,
    x=spec1.bin_indices,
    y=spec1.counts_vals,
    y_unc=spec1.counts_uncs,
    roi=(350, 450)
)

fitter.fit()

`custom_plot()` lets us see the fit result and diagnostics simultaneously

In [ ]:
fitter.custom_plot()
plt.show()

It looks like there's still a trend in the residuals. Let's try an exponential background model instead of linear.
Note we can pass a list of string model names to the Fitter.

In [ ]:
fitter = bq.Fitter(
    ['gauss', 'exp'],
    x=spec1.bin_indices,
    y=spec1.counts_vals,
    y_unc=spec1.counts_uncs,
    roi=(350, 450)
)
fitter.fit(backend='lmfit')

We can also specify that we want to plot residuals relative to the bin error (`residual_type=sigma`) or relative to the model (`residual_type=rel`) instead of absolute residuals (`residual_type=abs`, default).

In [ ]:
fitter.custom_plot(residual_type='sigma')
plt.show()

The normalized residuals help show that the exponential model fits fairly well---nearly all residuals are within $\pm2\sigma$, and there is no obvious trend.

We can also use the `iminuit` package to perform Poisson-loss fitting and also get uncertainties, a feature `lmfit` currently does not support.

In [ ]:
fitter.fit(
    backend='minuit',
    guess={'gauss_amp': 1000, 'gauss_mu': 400, 'gauss_sigma': 6, 'exp_amp': 7e7, 'exp_lam': -24},
    limits={'gauss_amp': (0, 1e10), 'gauss_mu': (0, 1000), 'gauss_sigma': (0, 1000), 'exp_amp': (0, None), 'exp_lam': (None, 0)}
)

In [ ]:
fitter.custom_plot(residual_type='sigma')
plt.show()

A better way to look at the `minuit` stats is with `fitter.result`. We can see that the fit parameters are valid, as are the error estimates.

In [ ]:
fitter.result